# 05Tools: Experiment Comparisons

Use the Python client to retrieve, review, and compare experiments from this series

### Prerequisites:
-  At least 1 of the notebooks in this series [05, 05a-05i]
    - Each of these notebooks creates an experiment.  Rerunning the notebooks creates additonal runs of the experiment.  The models created by each run are uploaded to the Vertex AI Model Registry with a version alias for the experiment run name and has labels for the experiment_name and run_name.
    
### Conceptual Flow & Workflow
<p align="center">
  <img alt="Conceptual Flow" src="../architectures/slides/05tools_exper_arch.png" width="45%">
&nbsp; &nbsp; &nbsp; &nbsp;
  <img alt="Workflow" src="../architectures/slides/05tools_exper_console.png" width="45%">
</p>

---
## Setup

inputs:

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [2]:
REGION = 'us-central1'
SERIES = '05'

packages:

In [3]:
from google.cloud import aiplatform
import pandas as pd
from IPython.display import HTML

clients:

In [4]:
aiplatform.init(project = PROJECT_ID, location = REGION)

---
## Experiments

Get a list of all experiments in this project:

In [5]:
experiments = aiplatform.Experiment.list()

Remove experiments not in the SERIES:

In [6]:
experiments = [e for e in experiments if e.name.split('-')[0:2] == ['experiment', SERIES]]

### Combine Runs from All Experiments in SERIES

Combine the runs from all experiments in SERIES into a single dataframe:

In [7]:
import time # temporary work around
results = []
for experiment in experiments:
        results.append(experiment.get_data_frame())
        print(experiment.name)
        time.sleep(60)
results = pd.concat(results)

experiment-05-05i-tf-classification-dnn
experiment-05-05h-tf-classification-dnn
experiment-05-05g-tf-classification-dnn
experiment-05-05f-tf-classification-dnn
experiment-05-05e-tf-classification-dnn
experiment-05-05d-tf-classification-dnn
experiment-05-05c-tf-classification-dnn
experiment-05-05b-tf-classification-dnn
experiment-05-05a-tf-classification-dnn
experiment-05-05-tf-classification-dnn


In [8]:
results

,experiment_name,run_name,run_type,state,param.var_split,param.epochs,param.series,param.experiment,param.hyperparameter.learning_rate,param.hyperparameterTuning.resource_name,...,time_series_metric.val_accuracy,time_series_metric.val_auprc,time_series_metric.val_loss,param.training.epochs,param.customJobs.display_name,param.trainingPipelines.resource_name,param.trainingPipelines.display_name,param.customJobs.resource_name,param.customJobs.link,param.customJobs.tensorboard
0,experiment-05-05i-tf-classification-dnn,run-20220927230444-18,system.ExperimentRun,COMPLETE,splits,10.0,05,05i,0.100000,projects/1026793852137/locations/us-central1/h...,...,0.999292,0.999667,0.004878,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,experiment-05-05i-tf-classification-dnn,run-20220927230444-17,system.ExperimentRun,COMPLETE,splits,10.0,05,05i,0.046467,projects/1026793852137/locations/us-central1/h...,...,0.999292,0.999714,0.004949,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,experiment-05-05i-tf-classification-dnn,run-20220927230444-16,system.ExperimentRun,COMPLETE,splits,10.0,05,05i,0.052617,projects/1026793852137/locations/us-central1/h...,...,0.999292,0.999575,0.004703,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,experiment-05-05i-tf-classification-dnn,run-20220927230444-15,system.ExperimentRun,COMPLETE,splits,10.0,05,05i,0.100000,projects/1026793852137/locations/us-central1/h...,...,0.999327,0.999577,0.004552,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,experiment-05-05i-tf-classification-dnn,run-20220927230444-14,system.ExperimentRun,COMPLETE,splits,10.0,05,05i,0.009142,projects/1026793852137/locations/us-central1/h...,...,0.999292,0.999763,0.004588,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,experiment-05-05-tf-classification-dnn,run-20220926162349,system.ExperimentRun,COMPLETE,splits,NaN,05,05,NaN,NaN,...,0.999256,0.999342,0.006727,4.0,NaN,NaN,NaN,NaN,NaN,NaN
3,experiment-05-05-tf-classification-dnn,run-20220827023100,system.ExperimentRun,COMPLETE,splits,NaN,05,05,NaN,NaN,...,0.999221,0.999388,0.005987,4.0,NaN,NaN,NaN,NaN,NaN,NaN
4,experiment-05-05-tf-classification-dnn,run-20220825175329,system.ExperimentRun,COMPLETE,splits,NaN,05,05,NaN,NaN,...,0.999221,0.999338,0.007228,4.0,NaN,NaN,NaN,NaN,NaN,NaN
5,experiment-05-05-tf-classification-dnn,run-20220825161109,system.ExperimentRun,COMPLETE,splits,NaN,05,05,NaN,NaN,...,0.999221,0.999391,0.005937,4.0,NaN,NaN,NaN,NaN,NaN,NaN


Get Links to all Experiment Runs with Vertex AI Training Jobs:
- Link to Job
- Link to TensorBoard for Job

In [9]:
# just names and training job columns
mask = results.columns.str.startswith(('param.trainingPipeline','param.customJobs','param.hyperparameterTuningJobs'))
mask[0:2] = True
HTML(results.loc[:,mask].to_html(render_links=True, escape=False))

,experiment_name,run_name,param.hyperparameterTuningJobs.display_name,param.customJobs.display_name,param.trainingPipelines.resource_name,param.trainingPipelines.display_name,param.customJobs.resource_name,param.customJobs.link,param.customJobs.tensorboard
0,experiment-05-05i-tf-classification-dnn,run-20220927230444-18,05_05i_20220928000517,NaN,NaN,NaN,NaN,NaN,NaN
1,experiment-05-05i-tf-classification-dnn,run-20220927230444-17,05_05i_20220928000517,NaN,NaN,NaN,NaN,NaN,NaN
2,experiment-05-05i-tf-classification-dnn,run-20220927230444-16,05_05i_20220928000517,NaN,NaN,NaN,NaN,NaN,NaN
3,experiment-05-05i-tf-classification-dnn,run-20220927230444-15,05_05i_20220928000517,NaN,NaN,NaN,NaN,NaN,NaN
4,experiment-05-05i-tf-classification-dnn,run-20220927230444-14,05_05i_20220928000517,NaN,NaN,NaN,NaN,NaN,NaN
5,experiment-05-05i-tf-classification-dnn,run-20220927230444-12,05_05i_20220928000517,NaN,NaN,NaN,NaN,NaN,NaN
6,experiment-05-05i-tf-classification-dnn,run-20220927230444-11,05_05i_20220928000517,NaN,NaN,NaN,NaN,NaN,NaN
7,experiment-05-05i-tf-classification-dnn,run-20220927230444-10,05_05i_20220928000517,NaN,NaN,NaN,NaN,NaN,NaN
8,experiment-05-05i-tf-classification-dnn,run-20220927230444-9,05_05i_20220928000517,NaN,NaN,NaN,NaN,NaN,NaN
9,experiment-05-05i-tf-classification-dnn,run-20220927230444-8,05_05i_20220928000517,NaN,NaN,NaN,NaN,NaN,NaN


### Hyperlinks To TensorBoard for Experiments and Comparison Of All Experiments

Get a hyperlink to the TensorBoard for each experiment:

In [10]:
tensorboards = []
for experiment in experiments:
    tboard = experiment.get_backing_tensorboard_resource().resource_name.replace('/', '+')
    tensorboards.append([experiment.name, f"https://{REGION}.tensorboard.googleusercontent.com/experiment/{tboard}+experiments+{experiment.name}"])
tensorboards = pd.DataFrame(tensorboards, columns = ['Experiment', 'TensorBoard Link'])    

In [11]:
HTML(tensorboards.to_html(render_links=True, escape=False))

,Experiment,TensorBoard Link
0,experiment-05-05i-tf-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7179142426307592192+experiments+experiment-05-05i-tf-classification-dnn
1,experiment-05-05h-tf-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7179142426307592192+experiments+experiment-05-05h-tf-classification-dnn
2,experiment-05-05g-tf-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7179142426307592192+experiments+experiment-05-05g-tf-classification-dnn
3,experiment-05-05f-tf-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7179142426307592192+experiments+experiment-05-05f-tf-classification-dnn
4,experiment-05-05e-tf-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7179142426307592192+experiments+experiment-05-05e-tf-classification-dnn
5,experiment-05-05d-tf-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7179142426307592192+experiments+experiment-05-05d-tf-classification-dnn
6,experiment-05-05c-tf-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7179142426307592192+experiments+experiment-05-05c-tf-classification-dnn
7,experiment-05-05b-tf-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7179142426307592192+experiments+experiment-05-05b-tf-classification-dnn
8,experiment-05-05a-tf-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7179142426307592192+experiments+experiment-05-05a-tf-classification-dnn
9,experiment-05-05-tf-classification-dnn,https://us-central1.tensorboard.googleusercontent.com/experiment/projects+1026793852137+locations+us-central1+tensorboards+7179142426307592192+experiments+experiment-05-05-tf-classification-dnn


Compare all experiments in SERIES with TensorBoard:

In [12]:
compare_link = f"https://{REGION}.tensorboard.googleusercontent.com/compare/"
for e, experiment in enumerate(experiments):
    if e>0: compare_link += ','
    tboard = experiment.get_backing_tensorboard_resource().resource_name.split('/')
    compare_link += f"{e+1}-{experiment.name}:{'+'.join(tboard[1::2])}+{experiment.name}"
print(compare_link)

https://us-central1.tensorboard.googleusercontent.com/compare/1-experiment-05-05i-tf-classification-dnn:1026793852137+us-central1+7179142426307592192+experiment-05-05i-tf-classification-dnn,2-experiment-05-05h-tf-classification-dnn:1026793852137+us-central1+7179142426307592192+experiment-05-05h-tf-classification-dnn,3-experiment-05-05g-tf-classification-dnn:1026793852137+us-central1+7179142426307592192+experiment-05-05g-tf-classification-dnn,4-experiment-05-05f-tf-classification-dnn:1026793852137+us-central1+7179142426307592192+experiment-05-05f-tf-classification-dnn,5-experiment-05-05e-tf-classification-dnn:1026793852137+us-central1+7179142426307592192+experiment-05-05e-tf-classification-dnn,6-experiment-05-05d-tf-classification-dnn:1026793852137+us-central1+7179142426307592192+experiment-05-05d-tf-classification-dnn,7-experiment-05-05c-tf-classification-dnn:1026793852137+us-central1+7179142426307592192+experiment-05-05c-tf-classification-dnn,8-experiment-05-05b-tf-classification-dnn:

## Rank Experiment Runs on a Metric

In [13]:
def ranker(metric = 'metric.test_auprc'):
    ranks = results[['experiment_name', 'run_name', metric]].copy().reset_index(drop = True)
    ranks['rank'] = ranks[metric].rank(method='dense', ascending = False)
    return ranks.sort_values(by = ['rank'])
    
ranks = ranker()

In [14]:
ranks

,experiment_name,run_name,metric.test_auprc,rank
125,experiment-05-05g-tf-classification-dnn,run-20220827031931-9,0.999813,1.0
35,experiment-05-05i-tf-classification-dnn,run-20220827031941-5,0.999812,2.0
53,experiment-05-05i-tf-classification-dnn,run-20220826194138-7,0.999812,3.0
139,experiment-05-05g-tf-classification-dnn,run-20220826184958-4,0.999766,4.0
73,experiment-05-05h-tf-classification-dnn,run-20220927230247-6,0.999765,5.0
...,...,...,...,...
76,experiment-05-05h-tf-classification-dnn,run-20220827031936-18,0.999302,139.0
14,experiment-05-05i-tf-classification-dnn,run-20220927230444-3,0.999297,140.0
42,experiment-05-05i-tf-classification-dnn,run-20220826194138-12,0.999237,141.0
55,experiment-05-05h-tf-classification-dnn,run-20220927230247-19,0.999222,142.0
